### Python Get and Post method

In [ ]:
import json
import wsgiref.simple_server
import urllib.parse
import requests

def application(environ, start_response):
    # requested path
    path = environ["PATH_INFO"]
    
    # requested method
    method = environ["REQUEST_METHOD"]
    
    # content type of response
    content_type = "text/html"

    if path == "/":
        if method == "POST":
            # getting wsgi.input obj
            input_obj = environ["wsgi.input"]
            
            # length of body
            input_length = int(environ["CONTENT_LENGTH"])
            
            # getting body of wsgi.input obj
            # decoding to string
            body = input_obj.read(input_length).decode()

            # parsing body of form
            data = urllib.parse.parse_qs(body, keep_blank_values=True)
            
            # Get Vehicle Type and Distance
            type = data["vehicle"]
            distance = float(data["distance"])
            
            if type == "car":
                choice = 1
            elif type == "motorcycle":
                choice = 2
                
            # connect with API and use
            MY_API_KEY="7SAZBY9HVTMZBTGQECC9J7SB0TAY"
            url = "https://beta3.api.climatiq.io/estimate"
            
            # set activity ID, region and year
            if choice == 1:
                activity_id = "passenger_vehicle-vehicle_type_automobile-fuel_source_na-distance_na-engine_size_na"
                region = "Global"
                year = "2017"
            elif choice == 2:
                activity_id = "passenger_vehicle-vehicle_type_motorcycle-fuel_source_na-engine_size_na-vehicle_age_na-vehicle_weight_na"
                region = "US"
                year = "2021"
            
            # set the parameters for retrieving emmissions estimation
            distance_unit = "mi"
            if choice == 1:
                distance = distance * 1.609  # convert to KM
                distance_unit = "km"
            parameters = {
                "distance": distance,
                "distance_unit": distance_unit
                }

            json_body = {
                "emission_factor": {
                    "activity_id": activity_id,
                    "region": region,
                },
                # Specify how much energy we're estimating for
                "parameters": parameters
            }
            
            # You must always specify your AUTH token in the "Authorization" header like this.
            authorization_headers = {"Authorization": f"Bearer: {MY_API_KEY}"}

            # This line will send the request and retrieve the body as JSON. - converted to python dict
            api_response = requests.post(url, json=json_body, headers=authorization_headers).json()

             # get Co2 emmisions
            co2e_gas = api_response["co2e"]
            
            # converting to JSON data
            response = json.dumps(co2e_gas).encode()
            status = "200 OK"
            
            # changing content-type
            content_type = "application/json"
            
        else:
            # reading html file
            with open("index.html","r") as f:
                response = f.read().encode()
            status = "200 OK"

    else:
        # 404 - path not found
        response = b"<h1>Not found</h1><p>Entered path not found</p>"
        status = "404 Not Found"

    # response headers
    headers = [
        ("Content-Type",content_type),
        ("Content-Length",str(len(response)))
    ]

    start_response(status, headers)
    return [response]
    

if __name__ == "__main__":
    w_s = wsgiref.simple_server.make_server(
        host="localhost",
        port=8021,
        app=application
    )
    w_s.serve_forever()